In [1]:
import bempp.api
import numpy as np

bempp.api.global_parameters.assembly.boundary_operator_assembly_type = 'dense'

In [2]:
hvec = np.linspace(0.08, 0.2, 15)
k_ext = 1
k = 1.5

In [3]:
e_error = []
m_error = []

ep_error = []
mp_error = []

In [4]:
def plane_wave_e(x, n, d, res):
    val =  np.array([np.exp(1j * k_ext * x[2]), 0, 0])
    res[:] = np.cross(val, n)
    
def plane_wave_m(x, n, d, res):
    val = np.array([0, 1., 0]) * np.exp(1j * k_ext * x[2])
    res[:] = np.cross(val, n)

In [ ]:
for h in hvec:
    grid = bempp.api.shapes.cube(h=h)
    multitrace = bempp.api.operators.boundary.maxwell.multitrace_operator(grid, k)
    ident = bempp.api.operators.boundary.sparse.multitrace_identity(grid, spaces='maxwell')
    calderon = .5 * ident + multitrace

    e = bempp.api.GridFunction(calderon.domain_spaces[0], 
                               dual_space=calderon.dual_to_range_spaces[0], fun=plane_wave_e)
    m = bempp.api.GridFunction(calderon.domain_spaces[1], 
                               dual_space=calderon.dual_to_range_spaces[1], fun=plane_wave_m)
    
    dof_count = calderon.domain_spaces[0].global_dof_count

    # Compute the error between C and C**2
    r1 = calderon * [e, m]
    r2 = calderon * r1
    electric_error = (r2[0] - r1[0]).l2_norm() / r1[0].l2_norm()
    magnetic_error = (r2[1] - r1[1]).l2_norm() / r1[1].l2_norm()
    e_error.append(electric_error)
    m_error.append(magnetic_error)

    # Compute the error of projecting a plane wave on the range of the discrete calderon operator
    
    [e_plane_wave_error, m_plane_wave_error] = (.5 * ident - multitrace) * [e, m]
    ep_error.append(e_plane_wave_error.l2_norm() / e.l2_norm())
    mp_error.append(m_plane_wave_error.l2_norm() / m.l2_norm())
    
    
    print('Mesh size: {0}'.format(h))
    print('Degrees of freedom: {0}'.format(dof_count))
    print("Electric error: {0}".format(electric_error))
    print("Magnetic error: {0}".format(magnetic_error))
    print("E-Plane Wave Error {0}".format(ep_error[-1]))
    print("M-Plane Wave Error {0}".format(mp_error[-1]))

In [ ]:
print(e_error)
print(m_error)
print(ep_error)
print(mp_error)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
slope_e, intercept_e = np.polyfit(np.log(hvec), np.log(e_error), 1)
slope_m, intercept_m = np.polyfit(np.log(hvec), np.log(m_error), 1)
slope_ep, intercept_ep = np.polyfit(np.log(hvec), np.log(ep_error), 1)
slope_mp, intercept_mp = np.polyfit(np.log(hvec), np.log(mp_error), 1)

print("Order of electric error: {0}".format(slope_e))
print("Order of magnetic error: {0}".format(slope_m))
print("Order of electric projection error: {0}".format(slope_ep))
print("Order of magnetic projection error: {0}".format(slope_mp))

In [ ]:
plt.loglog(hvec, e_error, 'b.-', markersize=10)
plt.loglog(hvec, m_error, 'r.-', markersize=10)
plt.legend(["electric error", "magnetic error"],fontsize = 16)

plt.xlabel('h', fontsize = 16)
plt.ylabel('relative error', fontsize = 16)
plt.tick_params(axis='both', which='major', labelsize=10)
plt.tick_params(axis='both', which='minor', labelsize=8)
